In [29]:
import os
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from tqdm import tqdm
import librosa
import numpy as np
import miditoolkit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, average_precision_score, accuracy_score
import random
import pretty_midi

from symusic import Score
from miditok import REMI, TokenizerConfig
from midiutil import MIDIFile
from glob import glob
# used chatgpt to help me generate some functions

In [30]:
# Processing the midi files
midi_files = glob('nes_midis/*')
print(len(midi_files))

config = TokenizerConfig(num_velocities=1)
tokenizer = REMI(config)
tokenizer.train(vocab_size = 2000, files_paths=midi_files)


2000





In [31]:
instruments = {}
bad_files = []

for file in midi_files:
    try:
        midi = pretty_midi.PrettyMIDI(file)
        for instrument in midi.instruments:
            name = pretty_midi.program_to_instrument_name(instrument.program)
            instruments[name] = instruments.get(name, 0) + 1
    except Exception as e:
        bad_files.append(file)

print(instruments)


{'Kalimba': 8, 'Marimba': 57, 'Music Box': 53, 'Drawbar Organ': 241, 'Orchestra Hit': 60, 'String Ensemble 1': 860, 'Choir Aahs': 124, 'Distortion Guitar': 238, 'Electric Bass (pick)': 201, 'Lead 8 (bass + lead)': 139, 'FX 5 (brightness)': 52, 'FX 3 (crystal)': 36, 'Trumpet': 214, 'String Ensemble 2': 175, 'Timpani': 100, 'Tremolo Strings': 124, 'Acoustic Guitar (nylon)': 187, 'Alto Sax': 67, 'Electric Bass (finger)': 408, 'Acoustic Grand Piano': 1613, 'FX 8 (sci-fi)': 25, 'Viola': 42, 'Reverse Cymbal': 17, 'Pad 1 (new age)': 52, 'Synth Strings 1': 103, 'Synth Strings 2': 32, 'Harpsichord': 152, 'Synth Bass 1': 131, 'Tubular Bells': 54, 'Lead 1 (square)': 947, 'Synth Bass 2': 114, 'Lead 2 (sawtooth)': 530, 'Lead 5 (charang)': 68, 'Clavinet': 115, 'Electric Guitar (clean)': 77, 'Synth Brass 1': 123, 'Pad 3 (polysynth)': 66, 'Electric Piano 1': 107, 'Flute': 176, 'Clarinet': 216, 'Pad 2 (warm)': 30, 'Gunshot': 34, 'Pan Flute': 32, 'Pizzicato Strings': 114, 'Acoustic Guitar (steel)': 164,